In [2]:
import numpy as np
import pandas as pd 
import os
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file = os.path.join(dirname, filename)

In [3]:
dir_ = Path('../input/vehicle-detection-image-set')
filepaths = list(dir_.glob(r'**/*.png'))
# Mapping the labels
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

# Paths & labels femalee eyes
filepaths = pd.Series(filepaths, name = 'File').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenating...
df = pd.concat([filepaths, labels], axis=1)

df = df.sample(frac = 1, random_state = 56).reset_index(drop = True)

In [28]:
df['Label'].value_counts()

In [29]:
plt.figure(figsize=(2,2))
a = plt.imread(df['File'][24])
plt.imshow(a)

In [30]:
plt.figure(figsize=(2,2))
b=plt.imread(df['File'][25])
plt.imshow(b)

In [31]:
fig,axes = plt.subplots(nrows=3,
                       ncols=3,
                       figsize=(10,10))

for i,ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df['File'][i]))
    ax.set_title(df['Label'][i])
plt.tight_layout()
plt.show()

In [32]:
from sklearn.model_selection import train_test_split
trainset_df, testset_df = train_test_split(df, train_size = 0.5, random_state=42)
display(trainset_df.head())
trainset_df.head()

In [34]:
LE=LabelEncoder()
y_test = LE.fit_transform(testset_df['Label'])

In [35]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.1,
                                    rotation_range = 20,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    validation_split = 0.1)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [36]:
training_set = train_datagen.flow_from_dataframe(
    dataframe = trainset_df,
    x_col = 'File',
    y_col = 'Label',
    target_size = (75,75),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
    seed = 2,
    subset = 'training')

validation_set = train_datagen.flow_from_dataframe(
    dataframe = trainset_df,
    x_col = 'File',
    y_col = 'Label',
    target_size = (75,75),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
    seed = 2,
    subset = 'validation')

test_set = test_datagen.flow_from_dataframe(
    dataframe = testset_df,
    x_col = 'File',
    y_col = 'Label',
    target_size = (75,75),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = False)

In [38]:
from tensorflow.keras.callbacks import EarlyStopping
cb = [EarlyStopping(monitor = 'loss', mode = 'min', patience = 5, restore_best_weights = True)]

In [39]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
CNN = InceptionV3(input_shape = (75, 75, 3), include_top = False, weights = 'imagenet')

In [40]:
for layer in CNN.layers:
    layer.trainable = False

In [41]:
x = layers.Flatten()(CNN.output)

In [44]:
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(1, activation='sigmoid')(x)

CNN = Model(CNN.input, x)

In [45]:
CNN.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
CNN.fit(training_set, epochs = 50, validation_data = validation_set, callbacks=cb)

With %10 test set only just after 5 epochs we reached the milestone of %98 accuracy.
With %50 test set only just after 5 epochs we reached the milestone of %98 accuracy.
